In [6]:
import auto_ml
from sklearn_pandas import DataFrameMapper, cross_val_score

import sklearn
import pandas as pd

In [3]:
def load_clean_transform(filename):
    data        = pd.read_csv(filename)
    file        = data.drop(["Name", "Ticket", "Cabin"], axis=1)
    file_age    = file["Age"].fillna(file["Age"].mean())
    file["Age"] = file_age
    
    return file

In [9]:
train = load_clean_transform("train.csv")
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [10]:
test = load_clean_transform("test.csv")
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [11]:
train["Embarked"] = train["Embarked"].fillna("S")
# data = no_name_train.groupby("Embarked")
# data.count() # `S` is in majority
train.count()

PassengerId    891
Survived       891
Pclass         891
Sex            891
Age            891
SibSp          891
Parch          891
Fare           891
Embarked       891
dtype: int64

In [12]:
test["Fare"] = test["Fare"].fillna(test["Fare"].mean())
test.count()

PassengerId    418
Pclass         418
Sex            418
Age            418
SibSp          418
Parch          418
Fare           418
Embarked       418
dtype: int64

In [14]:
transformer = DataFrameMapper([
    ("PassengerId", None),
    ("Sex", sklearn.preprocessing.LabelBinarizer()),
    ("Embarked", sklearn.preprocessing.LabelBinarizer()),
], df_out=True)

In [15]:
only_numeric_train = transformer.fit_transform(train.copy())
only_numeric_train.head()

,PassengerId,Sex,Embarked_C,Embarked_Q,Embarked_S
0,1,1,0,0,1
1,2,0,1,0,0
2,3,0,0,0,1
3,4,0,0,0,1
4,5,1,0,0,1


In [16]:
only_numeric_test = transformer.fit_transform(test)
only_numeric_test.head()

,PassengerId,Sex,Embarked_C,Embarked_Q,Embarked_S
0,892,1,0,1,0
1,893,0,0,0,1
2,894,1,0,1,0
3,895,1,0,0,1
4,896,0,0,0,1


In [17]:
df_train = train.merge(only_numeric_train, on="PassengerId").drop(["Embarked", "Sex_x", "PassengerId"], axis=1)
df_train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_y,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,1,0,0,1
1,1,1,38.0,1,0,71.2833,0,1,0,0
2,1,3,26.0,0,0,7.9250,0,0,0,1
3,1,1,35.0,1,0,53.1000,0,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,0,1


In [18]:
df_test = test.merge(only_numeric_test, on="PassengerId").drop(["Embarked", "Sex_x", "PassengerId"], axis=1)
df_test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_y,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,1,0,1,0
1,3,47.0,1,0,7.0000,0,0,0,1
2,2,62.0,0,0,9.6875,1,0,1,0
3,3,27.0,0,0,8.6625,1,0,0,1
4,3,22.0,1,1,12.2875,0,0,0,1


In [19]:
column_descriptions = {
    "Survived": "output",
    "Sex": "categorical",
    "Pclass": "categorical"
}

model = auto_ml.Predictor("classifier", column_descriptions=column_descriptions)

In [19]:
# df_train, fl_data = sklearn.model_selection.train_test_split(df_train, test_size=0.33)

In [20]:
model.train(df_train, optimize_final_model=True)

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at https://github.com/ClimbsRocks/auto_ml
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'warm_start': True, 'learning_rate': 0.1}
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'warm_start': True, 'learning_rate': 0.1}


********************************************************************************************
Optimizing the hyperparameters for your model now
About to run EvolutionaryAlgorithmSearchCV to find the optimal hyperparameters for the model GradientBoostingClass

[31] random_holdout_set_from_training_data's score is: -0.134
[31] random_holdout_set_from_training_data's score is: -0.214
[17] random_holdout_set_from_training_data's score is: -0.145
[24] random_holdout_set_from_training_data's score is: -0.147
[32] random_holdout_set_from_training_data's score is: -0.134
[33] random_holdout_set_from_training_data's score is: -0.132
[32] random_holdout_set_from_training_data's score is: -0.214
[34] random_holdout_set_from_training_data's score is: -0.133
[25] random_holdout_set_from_training_data's score is: -0.146
[18] random_holdout_set_from_training_data's score is: -0.148
[33] random_holdout_set_from_training_data's score is: -0.214
The number of estimators that were the best for this training dataset: 14
[26] random_holdout_set_from_training_data's score is: -0.144
The best score on the holdout set: -0.123105100442
[34] random_holdout_set_from_training_data's score is: -0.214
[19] random_holdout_set_from_training_data's score is: -0.153
[CV]  m

[64] random_holdout_set_from_training_data's score is: -0.211
[32] random_holdout_set_from_training_data's score is: -0.167
[8] random_holdout_set_from_training_data's score is: -0.162
[60] random_holdout_set_from_training_data's score is: -0.133
[66] random_holdout_set_from_training_data's score is: -0.21
[33] random_holdout_set_from_training_data's score is: -0.169
[62] random_holdout_set_from_training_data's score is: -0.131
[9] random_holdout_set_from_training_data's score is: -0.16
[34] random_holdout_set_from_training_data's score is: -0.17
[64] random_holdout_set_from_training_data's score is: -0.132
[35] random_holdout_set_from_training_data's score is: -0.17
[10] random_holdout_set_from_training_data's score is: -0.161
[36] random_holdout_set_from_training_data's score is: -0.17
[68] random_holdout_set_from_training_data's score is: -0.21
[11] random_holdout_set_from_training_data's score is: -0.161
[37] random_holdout_set_from_training_data's score is: -0.173
[12] random_hold

The number of estimators that were the best for this training dataset: 18
[127] random_holdout_set_from_training_data's score is: -0.127
The best score on the holdout set: -0.153234393153
[25] random_holdout_set_from_training_data's score is: -0.093
[CV]  model__max_features=None, model__learning_rate=0.1, model__loss=exponential, model__max_depth=7, _scorer=<auto_ml.utils_scoring.ClassificationScorer object at 0x7ff85fa0ce10>, model__n_estimators=125, model__subsample=0.5, score=-0.148614161338, total=   1.0s
[109] random_holdout_set_from_training_data's score is: -0.206
[130] random_holdout_set_from_training_data's score is: -0.127
[CV] model__max_features=None, model__learning_rate=0.1, model__loss=exponential, model__max_depth=2, _scorer=<auto_ml.utils_scoring.ClassificationScorer object at 0x7ff85fa0ce10>, model__n_estimators=75, model__subsample=0.8 
[133] random_holdout_set_from_training_data's score is: -0.127
[136] random_holdout_set_from_training_data's score is: -0.126
[26] 

[64] random_holdout_set_from_training_data's score is: -0.09
[35] random_holdout_set_from_training_data's score is: -0.093
[36] random_holdout_set_from_training_data's score is: -0.092
[18] random_holdout_set_from_training_data's score is: -0.153
[66] random_holdout_set_from_training_data's score is: -0.091
[37] random_holdout_set_from_training_data's score is: -0.09
[19] random_holdout_set_from_training_data's score is: -0.148
[68] random_holdout_set_from_training_data's score is: -0.089
[154] random_holdout_set_from_training_data's score is: -0.202
[38] random_holdout_set_from_training_data's score is: -0.09
[20] random_holdout_set_from_training_data's score is: -0.148
[70] random_holdout_set_from_training_data's score is: -0.089
The number of estimators that were the best for this training dataset: 40
The best score on the holdout set: -0.0867794200637
[39] random_holdout_set_from_training_data's score is: -0.089
[21] random_holdout_set_from_training_data's score is: -0.146
[CV]  mo

...

[86] random_holdout_set_from_training_data's score is: -0.199
[88] random_holdout_set_from_training_data's score is: -0.199
[90] random_holdout_set_from_training_data's score is: -0.197
[92] random_holdout_set_from_training_data's score is: -0.197
[94] random_holdout_set_from_training_data's score is: -0.197
[96] random_holdout_set_from_training_data's score is: -0.197
[98] random_holdout_set_from_training_data's score is: -0.196
[100] random_holdout_set_from_training_data's score is: -0.196
[103] random_holdout_set_from_training_data's score is: -0.196
[106] random_holdout_set_from_training_data's score is: -0.196
[109] random_holdout_set_from_training_data's score is: -0.196
[112] random_holdout_set_from_training_data's score is: -0.195
[115] random_holdout_set_from_training_data's score is: -0.195
[118] random_holdout_set_from_training_data's score is: -0.194
[121] random_holdout_set_from_training_data's score is: -0.193
[124] random_holdout_set_from_training_data's score is: -0.193

[6] random_holdout_set_from_training_data's score is: -0.228
[7] random_holdout_set_from_training_data's score is: -0.227
[8] random_holdout_set_from_training_data's score is: -0.225
[9] random_holdout_set_from_training_data's score is: -0.224
[10] random_holdout_set_from_training_data's score is: -0.222
[11] random_holdout_set_from_training_data's score is: -0.22
[12] random_holdout_set_from_training_data's score is: -0.219
[13] random_holdout_set_from_training_data's score is: -0.217
[14] random_holdout_set_from_training_data's score is: -0.216
[15] random_holdout_set_from_training_data's score is: -0.215
[16] random_holdout_set_from_training_data's score is: -0.213
[17] random_holdout_set_from_training_data's score is: -0.211
[18] random_holdout_set_from_training_data's score is: -0.21
[19] random_holdout_set_from_training_data's score is: -0.208
[20] random_holdout_set_from_training_data's score is: -0.207
[21] random_holdout_set_from_training_data's score is: -0.206
[22] random_ho

[330] random_holdout_set_from_training_data's score is: -0.135
[335] random_holdout_set_from_training_data's score is: -0.135
[340] random_holdout_set_from_training_data's score is: -0.135
[345] random_holdout_set_from_training_data's score is: -0.135
[350] random_holdout_set_from_training_data's score is: -0.135
[355] random_holdout_set_from_training_data's score is: -0.135
[360] random_holdout_set_from_training_data's score is: -0.135
[365] random_holdout_set_from_training_data's score is: -0.135
[370] random_holdout_set_from_training_data's score is: -0.135
[375] random_holdout_set_from_training_data's score is: -0.135
[380] random_holdout_set_from_training_data's score is: -0.135
[385] random_holdout_set_from_training_data's score is: -0.135
[390] random_holdout_set_from_training_data's score is: -0.135
[395] random_holdout_set_from_training_data's score is: -0.135
[400] random_holdout_set_from_training_data's score is: -0.135
[405] random_holdout_set_from_training_data's score is:

[18] random_holdout_set_from_training_data's score is: -0.188
[19] random_holdout_set_from_training_data's score is: -0.185
[20] random_holdout_set_from_training_data's score is: -0.184
[21] random_holdout_set_from_training_data's score is: -0.182
[22] random_holdout_set_from_training_data's score is: -0.182
[23] random_holdout_set_from_training_data's score is: -0.18
[24] random_holdout_set_from_training_data's score is: -0.179
[25] random_holdout_set_from_training_data's score is: -0.178
[26] random_holdout_set_from_training_data's score is: -0.176
[27] random_holdout_set_from_training_data's score is: -0.175
[28] random_holdout_set_from_training_data's score is: -0.174
[29] random_holdout_set_from_training_data's score is: -0.172
[30] random_holdout_set_from_training_data's score is: -0.171
[31] random_holdout_set_from_training_data's score is: -0.17
[32] random_holdout_set_from_training_data's score is: -0.169
[33] random_holdout_set_from_training_data's score is: -0.168
[34] rando

[380] random_holdout_set_from_training_data's score is: -0.088
[385] random_holdout_set_from_training_data's score is: -0.088
[390] random_holdout_set_from_training_data's score is: -0.089
[395] random_holdout_set_from_training_data's score is: -0.089
[400] random_holdout_set_from_training_data's score is: -0.089
[405] random_holdout_set_from_training_data's score is: -0.089
[410] random_holdout_set_from_training_data's score is: -0.089
[415] random_holdout_set_from_training_data's score is: -0.089
[420] random_holdout_set_from_training_data's score is: -0.089
[425] random_holdout_set_from_training_data's score is: -0.089
[430] random_holdout_set_from_training_data's score is: -0.089
[435] random_holdout_set_from_training_data's score is: -0.089
[440] random_holdout_set_from_training_data's score is: -0.089
[445] random_holdout_set_from_training_data's score is: -0.089
[450] random_holdout_set_from_training_data's score is: -0.089
[455] random_holdout_set_from_training_data's score is:

In [21]:
model.score(df_train, df_train["Survived"])

Here is our brier-score-loss, which is the default value we optimized for while training, and is the value returned from .score() unless you requested a custom scoring metric
It is a measure of how close the PROBABILITY predictions are.
0.0908

Here is the trained estimator's overall accuracy (when it predicts a label, how frequently is that the correct label?)
89.7%

Here is a confusion matrix showing predictions vs. actuals by label:
Predicted >    0    1  All
v Actual v                
0            529   20  549
1             72  270  342
All          601  290  891

Here is predictive value by class:
('Class: ', 0, '=', 0.8801996672212978)
('Class: ', 1, '=', 0.9310344827586207)
+----------------+-----------------------------------+--------------------------------+
| Bucket Edges   |   Predicted Probability Of Bucket |   Actual Probability of Bucket |
|----------------+-----------------------------------+--------------------------------|
| (0.051, 0.102] |                           

-0.090814334599907648

In [22]:
submission = pd.DataFrame({"PassengerId": [x for x in range(892, 892 + 418)], "Survived": model.predict(df_test)})

with open("submission_19.csv", "w") as f_ptr:
    f_ptr.write(submission.to_csv(index=False))